In [32]:
import pandas as pd

In [33]:
df = pd.read_excel('dados/Data e hora Ciclo 01.xlsx')
df

,id,uf,br,trecho,dt_levantamento,latitude,longitude,km,Ext. (km),sentido,Veículo,Lote,Empresa
0,8763503,BA,418,418BA000000,2020-07-05 08:01:33,-17.742603,-39.242783,0,0,C,IZV-6C04,2,Pavesys
1,8764282,BA,418,418BA000000,2020-07-05 08:01:33,-17.709105,-39.230339,4000,4,C,IZV-6C04,2,Pavesys
2,8765254,BA,418,418BA008454,2020-07-05 08:16:33,-17.665356,-39.228561,9000,0,C,IZV-6C04,2,Pavesys
3,8766219,BA,418,418BA008454,2020-07-05 08:16:33,-17.628445,-39.255634,14000,5,C,IZV-6C04,2,Pavesys
4,8768155,BA,418,418BA022013,2020-07-05 09:05:30,-17.634491,-39.338490,24000,0,C,IZV-6C04,2,Pavesys
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35130,42632036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.441893,-43.140396,10000,1,D,CEU-8E93,3,Consórcio Dynatest/VER
35131,42631036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.434614,-43.135315,11000,1,D,CEU-8E93,3,Consórcio Dynatest/VER
35132,42630036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.426563,-43.137203,12000,1,D,CEU-8E93,3,Consórcio Dynatest/VER
35133,42629036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.419632,-43.133453,13000,1,D,CEU-8E93,3,Consórcio Dynatest/VER


In [34]:
df = (df.drop(['Ext. (km)'], axis=1)
      .rename(columns={'Veículo':'veiculo', 'Lote':'lote', 'Empresa':'empresa'}))
df

,id,uf,br,trecho,dt_levantamento,latitude,longitude,km,sentido,veiculo,lote,empresa
0,8763503,BA,418,418BA000000,2020-07-05 08:01:33,-17.742603,-39.242783,0,C,IZV-6C04,2,Pavesys
1,8764282,BA,418,418BA000000,2020-07-05 08:01:33,-17.709105,-39.230339,4000,C,IZV-6C04,2,Pavesys
2,8765254,BA,418,418BA008454,2020-07-05 08:16:33,-17.665356,-39.228561,9000,C,IZV-6C04,2,Pavesys
3,8766219,BA,418,418BA008454,2020-07-05 08:16:33,-17.628445,-39.255634,14000,C,IZV-6C04,2,Pavesys
4,8768155,BA,418,418BA022013,2020-07-05 09:05:30,-17.634491,-39.338490,24000,C,IZV-6C04,2,Pavesys
...,...,...,...,...,...,...,...,...,...,...,...,...
35130,42632036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.441893,-43.140396,10000,D,CEU-8E93,3,Consórcio Dynatest/VER
35131,42631036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.434614,-43.135315,11000,D,CEU-8E93,3,Consórcio Dynatest/VER
35132,42630036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.426563,-43.137203,12000,D,CEU-8E93,3,Consórcio Dynatest/VER
35133,42629036,RJ,40,040RJ14058,2021-07-29 07:00:00,-22.419632,-43.133453,13000,D,CEU-8E93,3,Consórcio Dynatest/VER


In [35]:
df.loc[: ,'data'] = df.dt_levantamento.dt.date
df.loc[:, 'horario'] = df.dt_levantamento.dt.time.astype(object)
df = df.drop('dt_levantamento', axis=1)
df = df[['veiculo','data', 'horario','uf','trecho','km','sentido','lote']]
df

,veiculo,data,horario,uf,trecho,km,sentido,lote
0,IZV-6C04,2020-07-05,08:01:33,BA,418BA000000,0,C,2
1,IZV-6C04,2020-07-05,08:01:33,BA,418BA000000,4000,C,2
2,IZV-6C04,2020-07-05,08:16:33,BA,418BA008454,9000,C,2
3,IZV-6C04,2020-07-05,08:16:33,BA,418BA008454,14000,C,2
4,IZV-6C04,2020-07-05,09:05:30,BA,418BA022013,24000,C,2
...,...,...,...,...,...,...,...,...
35130,CEU-8E93,2021-07-29,07:00:00,RJ,040RJ14058,10000,D,3
35131,CEU-8E93,2021-07-29,07:00:00,RJ,040RJ14058,11000,D,3
35132,CEU-8E93,2021-07-29,07:00:00,RJ,040RJ14058,12000,D,3
35133,CEU-8E93,2021-07-29,07:00:00,RJ,040RJ14058,13000,D,3


In [36]:
a={}
veiculos = df.veiculo.unique()
sentidos = df.sentido.unique()

for veiculo in veiculos:
    x=[]
    for sentido in sentidos:
        subset = df.loc[(df.veiculo == veiculo) & (df.sentido == sentido)].copy()
        subset = subset.sort_values(by=['data','horario','uf','trecho','km']).reset_index(drop=True)
        x.append(subset)
    a[veiculo] = x



In [63]:
teste = a['CEU-8E93'][0].copy()

In [64]:
def dist_percorrida (df):
    df['distancia_percorrida'] = pd.Series(dtype='float64')
    df_index = df.index

    for i in df_index:
        if i == 0:
            df.loc[i,'distancia_percorrida'] = 0
        elif df.loc[i, 'data'] != df.loc[i-1, 'data']:
            df.loc[i,'distancia_percorrida'] = 0
        elif df.loc[i, 'km'] <= df.loc[i-1, 'km']:
            df.loc[i, 'distancia_percorrida'] = 0
        else:
            df.loc[i, 'distancia_percorrida'] = df.loc[i, 'km'] - df.loc[i-1, 'km']

    threshold = 2 * df['distancia_percorrida'].std()
    for i in df_index:
        if df.loc[i, 'distancia_percorrida'] >=  threshold:
            df.loc[i, 'distancia_percorrida'] = 0

    return  df

In [65]:
dist_percorrida(teste)
teste.to_csv('teste.csv')

In [59]:
teste.to_csv('teste.csv')

In [38]:
df_C = df.loc[(df['sentido'] == "C") & (df['veiculo'] == 'IZV-6C04')]
df_C = df_C.sort_values(by=['data','horario','uf','veiculo','trecho','km'])
df_C

,veiculo,data,horario,uf,trecho,km,sentido,lote
0,IZV-6C04,2020-07-05,08:01:33,BA,418BA000000,0,C,2
1,IZV-6C04,2020-07-05,08:01:33,BA,418BA000000,4000,C,2
2,IZV-6C04,2020-07-05,08:16:33,BA,418BA008454,9000,C,2
3,IZV-6C04,2020-07-05,08:16:33,BA,418BA008454,14000,C,2
4,IZV-6C04,2020-07-05,09:05:30,BA,418BA022013,24000,C,2
...,...,...,...,...,...,...,...,...
31825,IZV-6C04,2021-01-11,06:07:31,SE,101SE183873,185000,C,2
31826,IZV-6C04,2021-01-11,06:07:31,SE,101SE183873,186000,C,2
31827,IZV-6C04,2021-01-11,06:07:31,SE,101SE183873,187000,C,2
31828,IZV-6C04,2021-01-11,06:07:31,SE,101SE183873,188000,C,2


In [39]:
df = (df.groupby(['uf', 'veiculo', 'data', 'trecho', 'sentido'])['km']
 .apply(lambda x: x.max() - x.min())
 .reset_index())
df

,uf,veiculo,data,trecho,sentido,km
0,AC,QXH-5198,2020-09-18,317AC000000,C,39000
1,AC,QXH-5198,2020-09-19,317AC039607,C,9000
2,AC,QXH-5198,2020-09-19,317AC049308,C,20000
3,AC,QXH-5198,2020-09-19,317AC070434,C,20000
4,AC,QXH-5198,2020-09-19,317AC091637,C,6000
...,...,...,...,...,...,...
3378,TO,QXH-5198,2020-08-07,230TO083053,C,8000
3379,TO,QXH-5198,2020-08-07,230TO092640,C,7000
3380,TO,QXH-5198,2020-08-07,230TO100310,C,0
3381,TO,QXH-5198,2020-08-07,230TO101685,C,42000


In [40]:
pd.DataFrame(df.groupby(['uf', 'veiculo', 'data'])['km'].sum())

km
uf veiculo  data              
AC QXH-5198 2020-09-18   39000
            2020-09-19  248000
            2020-09-20  108000
            2020-09-21   31000
            2020-09-22  234000
...                        ...
TO QXH-5198 2020-08-03  171000
            2020-08-04  138000
            2020-08-05  129000
            2020-08-06  181000
            2020-08-07  172000

[523 rows x 1 columns]